<a href="https://colab.research.google.com/github/Chabon6/LegalTeachingAssistant/blob/main/%E3%80%90Advanced_RAG%E3%80%91Legal_Teaching_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Split Text Function

In [ ]:
!pip install --upgrade --quiet langchain langchain-openai langchain_community langchain_experimental
!pip install --upgrade --quiet langchain-chroma
!pip install langchain_core

In [6]:
import re
from langchain.schema import Document

class LawParser:
    def __init__(self, text_content):
        self.text_content = text_content
        self.law_name = self.extract_law_name()

    def extract_law_name(self):
        """
        提取文本中的法規名稱，格式為「法規名稱：XXX」。

        返回:
        str: 匹配到的法規名稱，如果沒有匹配則返回空字串。
        """
        pattern = r'法規名稱：(\w+)'
        match = re.search(pattern, self.text_content)
        if match:
            return match.group(1)
        else:
            print("未找到法規名稱")
            return None

    def extract_text_between_keywords(self, text, start_keyword, end_keyword, end=False):
        """
        從指定文本中，抓取兩個關鍵詞之間的文字塊。

        參數:
        - text: 要搜索的文本。
        - start_keyword: 開始關鍵詞。
        - end_keyword: 結束關鍵詞。
        - end: True or False 沒有end_keyword時填 True。

        返回:
        - 提取出的文字塊，若未找到匹配，返回 None。
        """
        if not end:
            pattern = rf"{re.escape(start_keyword)}(.+?){re.escape(end_keyword)}"
        else:
            pattern = rf"{re.escape(start_keyword)}(.+)$"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            print('未找到文字塊')
            return None

    def extract_provisions_names(self, text, provisions):
        """
        提取文本中的章節名稱，格式為「第 X 章(or節or目)...」的部分。

        參數:
        text (str): 要搜索的文本。
        provisions (str): 決定章or節or目

        返回:
        list: 匹配到的章、節、目名稱列表，若無匹配則返回空列表。
        """
        if provisions == '條':
            pattern = r'(第 \d+-?\w? 條)'
        else:
            pattern = rf'(第 \w+ {re.escape(provisions)}.+)'
        match = re.findall(pattern, text)
        if match:
            return match
        else:
            return []

    def provisions_split_list(self, text, provision):
        """
        根據章、節、目名稱，將文本分割成列表。

        參數:
        text (str): 要分割的文本。
        provision (str): 決定章or節or目

        返回:
        list: 分割後的章、節、目名稱列表。
        """
        splited_provisions_list = []
        names = self.extract_provisions_names(text, provision)
        provision_index = {'章': 1, '節': 2, '目': 3, '條': 4}.get(provision, 0)

        if names:
            pattern = r'\[.+\]'
            match = re.findall(pattern, text)

            for i, name in enumerate(names):
                if i != len(names) - 1:
                    extract_text = self.extract_text_between_keywords(text, name, names[i + 1])
                else:
                    extract_text = self.extract_text_between_keywords(text, name, "", end=True)
                splited_provisions_list.append(f"{match[0] if match else ''} [{provision_index}_{name}] " + extract_text)
        else:
            splited_provisions_list.append(text)
        return splited_provisions_list

    def law_split_list(self):
        """
        切出條為單位的chunks。

        返回:
        list: 切分後的章、節、目、條的列表。
        """
        Chapter_list = self.provisions_split_list(self.text_content, "章")
        Section_list = []
        for splited_text in Chapter_list:
            Section_list += self.provisions_split_list(splited_text, "節")

        Item_list = []
        for splited_text in Section_list:
            Item_list += self.provisions_split_list(splited_text, "目")

        final_list = []
        for splited_text in Item_list:
            final_list += self.provisions_split_list(splited_text, "條")
        return final_list

    def list2doc(self, final_list):
        """
        每一條文轉為 Langchain 的 Document。

        返回:
        list: 轉換後的 Document 列表。
        """
        for i, text in enumerate(final_list):
            source = re.findall(r'\[.+\]', text)[0]
            try:
                Chapter = re.findall(r'\[1.+?\]', text)[0]
            except:
                Chapter = ''
            try:
                Section = re.findall(r'\[2.+?\]', text)[0]
            except:
                Section = ''
            try:
                Item = re.findall(r'\[3.+?\]', text)[0]
            except:
                Item = ''
            try:
                Article = re.findall(r'\[4.+?\]', text)[0]
            except:
                Article = ''
            final_list[i] = Document(page_content=text, metadata={"law_name": self.law_name, "Chapter": Chapter, "Section": Section, "Item": Item, "Article": Article})
        return final_list

# Get Text

In [7]:
# 連結雲端
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/LangChain練習/RAG_LegalTeachingAssistant/Law_rtf'

Mounted at /content/drive


In [ ]:
!pip install striprtf

In [10]:
# 在file_path底下遍歷rtf檔
from striprtf.striprtf import rtf_to_text
import os


final_docs = []
for i, filename in enumerate(os.listdir(file_path)):
    if filename.endswith('.rtf'):
        print(f"Processing file: {filename}")
        # 讀取 RTF 文件
        with open(os.path.join(file_path, filename), 'r', encoding='utf-8') as file:
            rtf_content = file.read()
        # 轉換為純文本
        text_content = rtf_to_text(rtf_content)

        # 切割法典並轉成document
        parser = LawParser(text_content)
        final_list = parser.law_split_list()
        final_docs.append(parser.list2doc(final_list))
        print(f"Example: {final_docs[i][-1]}")
        print("Finished...")


Processing file: 公司法.rtf
Example: page_content='[1_第 九 章 附則] [4_第 449 條] 本法除中華民國八十六年六月二十五日修正公布之第三百七十三條及第三百八十三條、一百零四年七月一日修正公布之第五章第十三節條文、一百零七年七月六日修正之條文之施行日期由行政院定之，及九十八年五月二十七日修正公布之條文自九十八年十一月二十三日施行外，自公布日施行。' metadata={'law_name': '公司法', 'Chapter': '[1_第 九 章 附則]', 'Section': '', 'Item': '', 'Article': '[4_第 449 條]'}
Finished...
Processing file: 商業會計法.rtf
Example: page_content='[1_第 十 章 附則] [4_第 83 條] 1   本法自公布日施行。
2   本法中華民國一百零三年五月三十日修正之條文，自一百零五年一月一日施行。但商業得自願自一百零三年會計年度開始日起，適用中華民國一百零三年五月三十日修正之條文。' metadata={'law_name': '商業會計法', 'Chapter': '[1_第 十 章 附則]', 'Section': '', 'Item': '', 'Article': '[4_第 83 條]'}
Finished...


# RetrievalQA

### Package

In [11]:
# get API
from google.colab import userdata
# 文本 Embedding
from langchain_openai import OpenAIEmbeddings
# 向量儲存庫
from langchain_chroma import Chroma
# RAG
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
# 用於解析 LLM 輸出的基底
from langchain_core.output_parsers import BaseOutputParser
# 用於定義 prompt 模板
from langchain_core.prompts import PromptTemplate
# ger api
gpt_api = userdata.get('gpt_api')

from  langchain.chains import RetrievalQA

### Embedding

In [12]:
import os
persist_directory = '/content/drive/MyDrive/LangChain練習/RAG_LegalTeachingAssistant/Chroma_db'
# 有檔案就讀檔案
if len(os.listdir(persist_directory)) > 1:
    # 初始化 OpenAI 的 Embedding 模型
    gpt_api = userdata.get('gpt_api')
    embedding_model = OpenAIEmbeddings(openai_api_key=gpt_api)
    # 載入儲存在本地的 Chroma 資料庫
    chroma_db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
    print("已載入Chroma資料庫")
else:
    # 初始化 OpenAI 的 Embedding 模型
    embedding_model = OpenAIEmbeddings(openai_api_key=gpt_api)

    # Embedding 並儲存至向量儲存庫（另存一份到本地）
    chroma_db = Chroma.from_documents(final_docs, embedding_model, persist_directory=persist_directory)
    print("已儲存Chroma資料庫")

已載入Chroma資料庫


### Prompt

In [30]:
from langchain_core.prompts import PromptTemplate
# template
prompt_template = PromptTemplate(
                  input_variables = ["Quation", "Chunks"],
                  template = '''
                  <rule>你是一位專業的中華民國法律顧問，負責回答使用者的法律諮詢</rule>
                  <task>根據檢索的文本(chunks)回答問題，若你不知道答案就回答"不知道"，不要虛構。此外，應對每一選項附上參考來源，例如基於XX法XX章第X條，該選項正確(或錯誤)。</task>
                  <Quation>{Quation}</Quation>
                  <chunks>{Chunks}</chunks>
                  <format>
                  結論：
                  原因：
                  各選項所對應的完整法條：
                    (A)
                    (B)
                    (C)
                    (D)
                  </format>'''
                  )
# 設定使用者提問

Quation = '''
4 A 股份有限公司（下稱「A 公司」）為一家從事營建的非公開發行公司，該公司近期擬召開董事 會，討論公司投資東南亞事宜。依公司法之規定，下列敘述何者錯誤？
(A) A 公司過半數之董事得以書面記明提議事項及理由，請求董事長召集董事會。其請求提出後 15 日內，董事長不為召開時，過半數之董事得自行召集
(B) A 公司董事會之召集，除章程有較高之規定者外，應於 3 日前通知各董事及監察人
(C) A 公司章程得訂明經全體董事同意，董事就當次董事會議案以書面方式行使其表決權，而不實 際集會
(D) A 公司董事居住國外者，得以書面委託居住國內之其他股東，並向主管機關登記後，經常代理 出席董事會
'''

### Query Transformation(abandon)

In [ ]:
# # 將 LLM 的換行輸出轉換成 list
# from typing import List
# class LineListOutputParser(BaseOutputParser[List[str]]):
#     """Output parser for a list of lines."""
#     def parse(self, text: str) -> List[str]:
#         lines = text.strip().split("\n")
#         return list(filter(None, lines))  # Remove empty lines

# output_parser = LineListOutputParser()

# # 用於生成多個相似問題的 Prompt
# QUERY_PROMPT = PromptTemplate(
#     input_variables=["question"],
#     template="""You are an AI language model assistant. Your task is to generate five
#     different versions of the given user question to retrieve relevant documents from a vector
#     database. By generating multiple perspectives on the user question, your goal is to help
#     the user overcome some of the limitations of the distance-based similarity search.
#     Provide these alternative questions separated by newlines.
#     Original question: {question}""",
# )
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=gpt_api)

# # MultiQueryRetriever 的 Chain
# llm_chain = QUERY_PROMPT | llm | output_parser

# # 設定檢索器
# retriever = MultiQueryRetriever(
#     retriever=chroma_db.as_retriever(), llm_chain=llm_chain, parser_key="lines"
# )  # "lines" is the key (attribute name) of the parsed output

# # 檢索結果
# unique_docs = retriever.invoke(query)
# len(unique_docs)

# prompt: 接著將RAG得到的unique_docs與原始提問query一同交給LLM回答

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=True
# )

# result = qa_chain({"query": query})

# print(result["result"])
# print("Source Documents:")
# for document in result["source_documents"]:
#     print(document.page_content)
#     print("---")


### Small-to-Big

In [31]:
# 初始化檢索器
retriever = chroma_db.as_retriever()
# 設定檢索器返回多個段落
retriever.search_kwargs = {"k": 3}  # 設定檢索結果數量為 3
# 以使用者提問 Quation 為檢索參考而非 Prompt
unique_docs = retriever.invoke(Quation)

In [32]:
# 找出上一層的章or節or目
def get_targets(unique_docs):
    targets = {}
    for doc in unique_docs:
        for metadata_key in ['Item', 'Section', 'Chapter']:
            if doc.metadata[metadata_key] != '':
                try:
                  targets[metadata_key].add(doc.metadata[metadata_key])
                except:
                  targets[metadata_key] = set()
                  targets[metadata_key].add(doc.metadata[metadata_key])
                break
    return targets

In [33]:
targets = get_targets(unique_docs)
# 把上一層的所有條文全部索引出來
total_chunks = ''
for target in targets.keys():
    for i in targets[target]:
        total_chunks += str(chroma_db.get(where={target: i})['documents'])

In [34]:
# 初始化 OpenAI 的 LLM 模型
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=gpt_api)

# 提示詞
prompt = prompt_template.format(Quation=Quation, Chunks=total_chunks)

#
result = llm.invoke(prompt)

In [35]:
print(result.content)

結論：  
選項 (D) 錯誤。

原因：  
根據公司法的規定，董事居住國外者，並不允許以書面委託居住國內的其他股東，並向主管機關登記後，經常代理出席董事會。董事會的出席應由董事親自出席，或依章程規定由其他董事代理出席，而非股東。

各選項所對應的完整法條：  
(A) 根據公司法第 203-1 條第 2 項，過半數之董事得以書面記明提議事項及理由，請求董事長召集董事會。其請求提出後 15 日內，董事長不為召開時，過半數之董事得自行召集。該選項正確。  

(B) 根據公司法第 204 條第 1 項，董事會之召集，除章程有較高之規定者外，應於 3 日前通知各董事及監察人。該選項正確。  

(C) 根據公司法第 205 條第 5 項，公司章程得訂明經全體董事同意，董事就當次董事會議案以書面方式行使其表決權，而不實際集會。該選項正確。  

(D) 根據公司法第 205 條第 1 項，董事會開會時，董事應親自出席。但公司章程訂定得由其他董事代理者，不在此限。並無規定董事可以委託股東代理出席董事會。該選項錯誤。
